In [2]:
import pandas  as pd 
from itertools import product
from collections import defaultdict
from itertools import groupby
from copy import deepcopy
import os
import csv
import ast
import re

Estado actual: 
* Pandas es muy lento para lo que quiero hacer
* Trabajo actual: separar los desempates en: Primero wins, despues rounds, despues mapas (probablemente no se implemente)
* Potenciales soluciones: Cambiar la manera de armar la tabla a Numpy (deberia ser lo mas rapido por mucho) Ventajas: velocidad y puedo evaluar por filas los resultados 

-------------------------------------------------------------------------

Arranco casi de 0, problemas con la unicacion de memoria de los objetos y se armo un re quilombo. 
Intento 2 diseño enfocado tabla final que tengo y estructurar desde ese punto hacia atras. 

-------------------------------------------------------------------------

24-1-25
Funcionando en trabajo por lotes. 
Dificultades actuales ordenar los batch para procesarlos. Resto de las funcionalidades parecen ok 


In [3]:
#Esto es raro, siento que estoy dando vueltas en circulo con este diccionario creando los objetos y volviendo a usarlos 
#Lo termino usando con un zip, lo que puede ser peligroso con el orden y las keys
#Agregar h2h ganados aca, voy a tener que modificar como funcionan las fuciones 
teamNameScores = {
                "name": ["Exolos","6K","FUS","AB3","CHV","Contra","FUEGO","Cinta Negra"],
                "wins": [5,4,4,3,2,1,1,0],
                "loss":[0,1,1,2,3,4,4,5],
                "matchWins" : [10,8,8,7,6,3,2,0],
                "matchLoss" :  [0,2,4,5,7,8,8,10],
                "roundWin":[130,123,129,132,130,104,88,56],
                "roundLoss":[44,94,112,116,139,137,120,130],
                'h2h' : [['FUEGO','Cinta Negra','6K','CHV','FUS'],
                         ['FUEGO','Cinta Negra','AB3','Contra'],
                         ['AB3','CHV','Contra','Cinta Negra'],
                         ['Cinta Negra','Contra','CHV'],
                         ['Contra','FUEGO'],
                         ['FUEGO'],
                         ['Cinta Negra'],
                         []]
                  }

roundMatches = {
                "week5" : [["FUEGO","FUS"], ["AB3","Exolos"],["CHV","6K"], ["Contra","Cinta Negra"]],
                "week6" : [["CHV","Cinta Negra"], ["6K","FUS"],["Contra","Exolos"], ["FUEGO","AB3"]]
                }

In [3]:
# Entry vacia para checkear creacion
teamNameScores = {
    "name": ["Exolos", "6K", "FUS", "AB3", "CHV", "Contra", "FUEGO", "Cinta Negra"],
    "wins": [0, 0, 0, 0, 0, 0, 0, 0, 0, 00, 0, 0],
    "loss": [0, 0, 0, 0, 0, 0, 0, 0, 0, 00, 0, 0],
    "matchWins": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "matchLoss": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "roundWin": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "roundLoss": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
}

roundMatches = {
                "week5" : [["FUEGO","FUS"], ["AB3","Exolos"],["CHV","6K"], ["Contra","Cinta Negra"]],
                "week6" : [["CHV","Cinta Negra"], ["6K","FUS"],["Contra","Exolos"], ["FUEGO","AB3"]]
                }

In [4]:
def process_for_round(teams):
    previous_results = []
    for i, team in enumerate(teams["name"]):
        temp_dict = {
            "UP": 1,
            "Team": team,
            "wins": teams["wins"][i],
            "matchDelta": teams["matchWins"][i] - teams["matchLoss"][i],
            "h2h": teams['h2h'][i]
        }
        previous_results.append(temp_dict)
    return previous_results

In [5]:
def run_round_delta_with_resume(matches, previous_results, batch_size=512, output_file="results_part", progress_file="progress.txt"):
    match_outcomes = list(product([0, 1, 2, 3], repeat=len(matches)))
    result_mapping = {
        0: (0, 2),
        1: (1, 2),
        2: (2, 1),
        3: (2, 0),
    }

    test_group = defaultdict(list)
    for result in previous_results:
        test_group[result["UP"]].append(result)

    # Leer el progreso guardado
    start_batch = 0
    if os.path.exists(progress_file):
        with open(progress_file, "r") as f:
            start_batch = int(f.read().strip())

    total_batches = len(match_outcomes) // batch_size + 1

    for batch in range(start_batch, total_batches):
        batch_outcomes = match_outcomes[batch * batch_size: (batch + 1) * batch_size]
        new_results = []

        for up, group in test_group.items():
            simul_teams = {
                team_data["Team"]: {
                    "wins": team_data["wins"],
                    "matchDelta": team_data["matchDelta"],
                    'h2h': team_data['h2h']
                }
                for team_data in group
            }

            for combination in batch_outcomes:
                temp_teams = deepcopy(simul_teams)

                for match_index, result in enumerate(combination):
                    team1, team2 = matches[match_index]
                    team1_score, team2_score = result_mapping[result]
                    delta_score = abs(team1_score - team2_score)

                    if result in [2, 3]:
                        temp_teams[team1]["wins"] += 1
                        temp_teams[team1]["matchDelta"] += delta_score
                        temp_teams[team2]["matchDelta"] -= delta_score
                        temp_teams[team1]["h2h"] += [team2]
                    else:
                        temp_teams[team2]["wins"] += 1
                        temp_teams[team2]["matchDelta"] += delta_score
                        temp_teams[team1]["matchDelta"] -= delta_score
                        temp_teams[team2]["h2h"] += [team1]

                new_up = f"{up}-{batch}-{combination}"

                for team, data in temp_teams.items():
                    new_results.append({
                        "UP": new_up,
                        "Team": team,
                        "wins": data["wins"],
                        "matchDelta": data["matchDelta"],
                        'h2h': data['h2h']
                    })

        # Guardado resultados parciales
        output_file_batch = output_file+str(batch)+'.csv'
        df = pd.DataFrame(new_results)
        file_exists = os.path.exists(output_file_batch) and os.path.getsize(output_file_batch) > 0
        df.to_csv(output_file_batch, mode='a', index=False, header=not file_exists)

        # Guardado del progreso
        with open(progress_file, "w") as f:
            f.write(str(batch + 1))

        print(f"Lote {batch + 1}/{total_batches} procesado y guardado.")

    print("Procesamiento completado.")
    os.remove(progress_file) #borrar el progreso si termino de procesar todo 

In [6]:
def groupby_UP(matchresults) -> list:
    group_result = defaultdict(list)
    for result in matchresults:
        group_result[result["UP"]].append(result)
    return group_result


def group_table(teamDict) -> list:
    return [list(group) for _, group in groupby(teamDict, key=lambda x: x[1]["wins"])]


def heriarchy_dict_for_sort(groupby_up_dict: dict) -> dict:
    hierarchy_dict_to_return = {}
    for key in groupby_up_dict.keys():
        hierarchy_dict_to_append = {}
        for team in groupby_up_dict[key]:
            hierarchy_dict_to_append[team["Team"]] = {
                "wins": team["wins"],
                "matchDelta": team["matchDelta"],
                "h2h": team["h2h"],
            }
        hierarchy_dict_to_return[key] = hierarchy_dict_to_append
    return hierarchy_dict_to_return


def sort_dict_by_wins(single_up_round) -> list:
    count = 1  # Esto se puede sacar ?
    sorted_table = sorted(
        single_up_round.items(),
        key=lambda x: (x[1]["wins"], x[1]["matchDelta"]),
        reverse=True,
    )
    grouped_table = group_table(sorted_table)
    count += 1  # Esto se puede sacar ?
    return grouped_table


# Tiebreak Rules
def two_teams_tie(
    tie_teams: list,
) -> (
    list
):  # Esta es rara, falla en tablas incompletas (pueden no haber jugado los dos equipos, pero no deberia falla en la ultima ronda)
    if len(tie_teams) != 2:
        print("not 2 tie teams")
    else:
        team1_h2h_vs_rival = set(tie_teams[0][1]["h2h"]) & set([tie_teams[1][0]])

        if len(team1_h2h_vs_rival) == 0:
            tie_teams = tie_teams[1:] + tie_teams[:1]
    return tie_teams


def three_way_tie(tie_teams: list) -> list:
    if len(tie_teams) != 3:
        print("not 3 way tie")
    else:
        teams = [x[0] for x in tie_teams]
        for index, team in enumerate(tie_teams):  # one team superiority vs the other 2
            team_h2h = set(tie_teams[index][1]["h2h"]) & set(teams)

            if len(team_h2h) == 2:
                pop_team = tie_teams.pop(index)
                remaining_tie = two_teams_tie(tie_teams)
                tie_teams = [pop_team] + remaining_tie

            elif index == 2:
                print("Round diff needed")
    return tie_teams


def tiebreaker(tie_teams):
    amount_tie_teams = len(tie_teams)
    if amount_tie_teams >= 4:
        match_tie_set = set()
        for team in tie_teams:  # This need to be check
            match_tie_set.add(team[1]["matchDelta"])
        if amount_tie_teams != match_tie_set:
            pass

    if amount_tie_teams == 3:
        tie_teams = three_way_tie(tie_teams)

    if amount_tie_teams == 2:  # Esto se puede cambiar y modificar solo la posición
        tie_teams = two_teams_tie(tie_teams)

    return tie_teams or []

In [7]:
def untie_round_save(up_dict_sorted,dict_up_position, key):
    positon = 1
    return_dict = []
    for list_team in up_dict_sorted:
        if len(list_team) == 1:                  
            dict_up_position[list_team[0][0]][str(positon)][0] += 1
            dict_up_position[list_team[0][0]][str(positon)][1].add(key)
            positon += 1
            return_dict += [list_team]
        if len(list_team) > 1:           
            sorted_dict_temp = tiebreaker(list_team)
            for team in sorted_dict_temp:                           
                dict_up_position[team[0]][str(positon)][0] += 1
                dict_up_position[team[0]][str(positon)][1].add(key)
                positon += 1
            return_dict += sorted_dict_temp          
    return return_dict

In [8]:
def transform_csv_2_dict(url):
    with open(url, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        data = []
        for row in reader:
            data.append({
                "UP": row["UP"], 
                "Team": row["Team"],
                "wins": int(row["wins"]),
                "matchDelta": int(row["matchDelta"]),
                "h2h": ast.literal_eval(row["h2h"])  
            })
    return data

In [9]:
test_set_dict = {}
for team in teamNameScores["name"]:
    test_set_dict[team] = {
        "1": [0,set()],
        "2": [0,set()],
        "3": [0,set()],
        "4": [0,set()],
        "5": [0,set()],
        "6": [0,set()],
        "7": [0,set()],
        "8": [0,set()],
    }

In [10]:
def process_batch(feed_path, result_path, matches, batch_size=512, output_file="result.csv", progress_file="progress.txt"): #Ojo que ouput result no se esta usando
    # Listar todos los archivos CSV en el directorio
    files_csv = [archivo for archivo in os.listdir(feed_path) if archivo.endswith(".csv")]

    try:
        os.mkdir(result_path)
        print(f"Directory '{result_path}' created successfully.")
    except FileExistsError:
        print(f"Directory '{result_path}' already exists.")
    except PermissionError:
        print(f"Permission denied: Unable to create '{result_path}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

    for file in files_csv:
        file_path = os.path.join(feed_path, file)  
        print(f"Processing file: {file_path}")
        
        try:
            data = transform_csv_2_dict(file_path)
            file_output = os.path.join(result_path, f"processed_{file}")
            
            run_round_delta_with_resume(
                matches=matches,
                previous_results=data,
                batch_size=batch_size,
                output_file=file_output,
                progress_file=progress_file
            )
            
            print(f"File {file} processed and saved to {file_output}.")
        
        except Exception as e:
            print(f"Error processing file {file}: {e}")

In [16]:
teams = teamNameScores
matches = (("FUEGO","FUS"), ("AB3","Exolos"),("CHV","6K"), ("Contra","Cinta Negra"))
matches2 = (("CHV","Cinta Negra"), ("6K","FUS"),("Contra","Exolos"), ("FUEGO","AB3"))

In [17]:
#Process de fecha 1 (se pueden mejorar los paths)
previous_results = process_for_round(teams)

run_round_delta_with_resume(matches, previous_results=previous_results, batch_size=512, output_file=".//results//results_batch_test", progress_file=".//results//progress//progress1.txt")

Lote 1/1 procesado y guardado.
Procesamiento completado.


In [18]:
#Aca va match2
feed_path = './/results'
path_results = './/results//fecha_2'

process_batch(feed_path,path_results, matches2, batch_size=512,output_file="result_test.csv",progress_file="progress.txt")

Directory './/results//fecha_2' already exists.
Processing file: .//results\results_batch_test0.csv
Lote 1/1 procesado y guardado.
Procesamiento completado.
File results_batch_test0.csv processed and saved to .//results//fecha_2\processed_results_batch_test0.csv.


In [19]:
#procesar lote de resultados: 
csv_to_process = './/results//fecha_2'

files_csv = [archivo for archivo in os.listdir(csv_to_process) if archivo.endswith(".csv")]

In [20]:
test_set_dict

{'Exolos': {'1': [1, {'1-0-(0, 0, 0, 0)-0-(0, 0, 0, 0)'}],
  '2': [0, set()],
  '3': [0, set()],
  '4': [0, set()],
  '5': [0, set()],
  '6': [0, set()],
  '7': [0, set()],
  '8': [0, set()]},
 '6K': {'1': [0, set()],
  '2': [0, set()],
  '3': [1, {'1-0-(0, 0, 0, 0)-0-(0, 0, 0, 0)'}],
  '4': [0, set()],
  '5': [0, set()],
  '6': [0, set()],
  '7': [0, set()],
  '8': [0, set()]},
 'FUS': {'1': [0, set()],
  '2': [1, {'1-0-(0, 0, 0, 0)-0-(0, 0, 0, 0)'}],
  '3': [0, set()],
  '4': [0, set()],
  '5': [0, set()],
  '6': [0, set()],
  '7': [0, set()],
  '8': [0, set()]},
 'AB3': {'1': [0, set()],
  '2': [0, set()],
  '3': [0, set()],
  '4': [0, set()],
  '5': [0, set()],
  '6': [0, set()],
  '7': [0, set()],
  '8': [0, set()]},
 'CHV': {'1': [0, set()],
  '2': [0, set()],
  '3': [0, set()],
  '4': [0, set()],
  '5': [0, set()],
  '6': [0, set()],
  '7': [0, set()],
  '8': [0, set()]},
 'Contra': {'1': [0, set()],
  '2': [0, set()],
  '3': [0, set()],
  '4': [0, set()],
  '5': [0, set()],
  '

In [21]:
for file in files_csv:
    file_path = csv_to_process + '//' + file
    data = transform_csv_2_dict(file_path)
    grouped_data = groupby_UP(data)
    dict_ready_to_sorot = heriarchy_dict_for_sort(grouped_data)
    for key in dict_ready_to_sorot.keys():
        sorted_dict = sort_dict_by_wins(dict_ready_to_sorot[key])
        untie_round_save(sorted_dict,test_set_dict,key)

Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff

In [22]:
#Creo que esto no haace falta en los test
path_file = csv_to_process + '//' + files_csv[0]
data = transform_csv_2_dict(path_file)

In [23]:
groupby_up = groupby_UP(data)

In [24]:
dict_ready_to_sort = heriarchy_dict_for_sort(groupby_up)

In [31]:
dict_ready_to_sort

{'1-0-(0, 0, 0, 0)-0-(0, 0, 0, 0)': {'Exolos': {'wins': 7,
   'matchDelta': 14,
   'h2h': ['FUEGO', 'Cinta Negra', '6K', 'CHV', 'FUS', 'AB3', 'Contra']},
  '6K': {'wins': 5,
   'matchDelta': 6,
   'h2h': ['FUEGO', 'Cinta Negra', 'AB3', 'Contra', 'CHV']},
  'FUS': {'wins': 6,
   'matchDelta': 8,
   'h2h': ['AB3', 'CHV', 'Contra', 'Cinta Negra', 'FUEGO', '6K']},
  'AB3': {'wins': 4,
   'matchDelta': 3,
   'h2h': ['Cinta Negra', 'Contra', 'CHV', 'FUEGO']},
  'CHV': {'wins': 2, 'matchDelta': -6, 'h2h': ['Contra', 'FUEGO']},
  'Contra': {'wins': 1, 'matchDelta': -12, 'h2h': ['FUEGO']},
  'FUEGO': {'wins': 1, 'matchDelta': -12, 'h2h': ['Cinta Negra']},
  'Cinta Negra': {'wins': 2, 'matchDelta': -6, 'h2h': ['Contra', 'CHV']}},
 '1-0-(0, 0, 0, 0)-0-(0, 0, 0, 1)': {'Exolos': {'wins': 7,
   'matchDelta': 14,
   'h2h': ['FUEGO', 'Cinta Negra', '6K', 'CHV', 'FUS', 'AB3', 'Contra']},
  '6K': {'wins': 5,
   'matchDelta': 6,
   'h2h': ['FUEGO', 'Cinta Negra', 'AB3', 'Contra', 'CHV']},
  'FUS': {'wins

In [25]:
def sort_dict_by_wins(single_up_round):
    #count = 1
    sorted_table = sorted(single_up_round.items(), key =lambda x: (x[1]["wins"], x[1]["matchDelta"]), reverse=True)
    
    grouped_table = group_table(sorted_table)
    
    #count += 1
    return grouped_table
#Aca tengo que agregar el desempate y los resultados y no crear todo la lista larga? 

In [26]:
def two_teams_tie(tie_teams):
    if len(tie_teams) != 2:
        print("not 2 tie teams")
    else:
        team1_h2h_vs_rival = set(tie_teams[0][1]["h2h"]) & set([tie_teams[1][0]])
        if len(team1_h2h_vs_rival) == 0:
            tie_teams = tie_teams[1:] + tie_teams[:1]

    return tie_teams

In [27]:
def three_way_tie(tie_teams):
    if len(tie_teams) != 3:
        print("not 3 way tie")
    else:
        teams = [x[0] for x in tie_teams]
        for index, team in enumerate(tie_teams):  # one team superiority vs the other 2
            team_h2h = set(tie_teams[index][1]["h2h"]) & set(teams)

            if len(team_h2h) == 2:
                pop_team = tie_teams.pop(index)
                remaining_tie = two_teams_tie(tie_teams)
                tie_teams = [pop_team] + remaining_tie

            elif index == 2:
                pass#print("Round diff needed")

    return tie_teams

In [28]:
def tiebreaker(tie_teams):
    amount_tie_teams = len(tie_teams)
    if amount_tie_teams >= 4:
        match_tie_set = set()
        for team in tie_teams: #This need to be check
            match_tie_set.add(team[1]["matchDelta"])  
        if amount_tie_teams != match_tie_set:
            pass 
        
    if amount_tie_teams == 3:
        tie_teams = three_way_tie(tie_teams)
        
    if amount_tie_teams == 2:  # Esto se puede cambiar y modificar solo la posición
        
        tie_teams = two_teams_tie(tie_teams)
        
    return tie_teams or []

In [29]:
def untie(up_dict_sorted,dict_up_position):
    positon = 1
    return_dict = []
    for list_team in up_dict_sorted:
        if len(list_team) == 1:            
            dict_up_position[list_team[0][0]][str(positon)] += 1
            positon += 1
            return_dict += [list_team]
        if len(list_team) > 1:           
            sorted_dict_temp = tiebreaker(list_team)
            for team in sorted_dict_temp:             
                dict_up_position[team[0]][str(positon)] += 1
                positon += 1
            return_dict += sorted_dict_temp          
    return return_dict

In [30]:
def extract_parentheses_values(input_string):
    # Valores dentro de parentesis
    pattern = r'\((.*?)\)'
    matches = re.findall(pattern, input_string)
    # Convertir los grupos grupo capturado en una lista de enteros
    results = [list(map(int, match.split(','))) for match in matches]
    return results

In [31]:
def create_nested_dict(start, end,matches_per_week=4):
    match_results = {}
    for i in range(start, end + 1):
        match_results[str(i)] = {str(j): [] for j in range(matches_per_week)} # ACA OJO CON EL 4 HARDCODEADO
    return match_results

# Crear el diccionario desde 1 hasta 10 como ejemplo
matchs_per_week = 4
nested_dict = create_nested_dict(1, 8,matchs_per_week)


In [51]:
nested_dict

{'1': {'0': [], '1': [], '2': [], '3': []},
 '2': {'0': [], '1': [], '2': [], '3': []},
 '3': {'0': [], '1': [], '2': [], '3': []},
 '4': {'0': [], '1': [], '2': [], '3': []},
 '5': {'0': [], '1': [], '2': [], '3': []},
 '6': {'0': [], '1': [], '2': [], '3': []},
 '7': {'0': [], '1': [], '2': [], '3': []},
 '8': {'0': [], '1': [], '2': [], '3': []}}

In [32]:
#Probablemente aca me conviene pasar a sql, armar la base de datos y trabajarla desde sql directo
#Checkear que es mas rapido, correr 2 veces el for con un extracted element particular o el enumarte de extracted_element + mas el otro for
#El enumarated(extracted_element) con el for evaluando los 0 1 es mas escalable 

def create_position_dicts(team, position_set_dict,nested_dict_range=[1,4],final_poss=4):
    result_mapping = {
        0: '0-2',
        1: '1-2',
        2: '2-1',
        3: '2-0',
    }
    nested_dict = create_nested_dict(nested_dict_range[0], nested_dict_range[1])
    nested_dict_match2 = create_nested_dict(nested_dict_range[0], nested_dict_range[1])
    for position in position_set_dict[team]:
        if int(position) < final_poss:
            for element in position_set_dict[team][position][1]:
                extracted_element = extract_parentheses_values(element)
                for number, result in enumerate(extracted_element[0]): 
                    nested_dict[str(position)][str(number)].append(
                        (matches[number][0], result_mapping[result], matches[number][1])
                    )
                for number, result in enumerate(extracted_element[1]):
                    nested_dict_match2[str(position)][str(number)].append(
                        (matches2[number][0], result_mapping[result], matches2[number][1])
                    )
    return nested_dict, nested_dict_match2

In [ ]:
def df_with_positions(weeek1,week2,team):
    postion = ['position','team']
    week1columns = ['week1_match_1','week1_match_2','week1_match_3','week1_match_4']
    week2columns = ['week2_match_1','week2_match_2','week2_match_3','week2_match_4']

    columns_df = postion + week1columns + week2columns

    df_base = pd.DataFrame(columns=columns_df)

    for key in weeek1.keys():
        
        df_week1 = pd.DataFrame.from_dict(weeek1[key])
        
        df_week1.columns = ['week1_match_1','week1_match_2','week1_match_3','week1_match_4']
        df_week2 = pd.DataFrame.from_dict(week2[key])
        df_week2.columns = ['week2_match_1','week2_match_2','week2_match_3','week2_match_4']
        
        df_combined = pd.concat([df_week1, df_week2], axis=1)
        df_combined['position'] = key
        df_combined['team'] = team
       

        df_base = pd.concat([df_base,df_combined],ignore_index=True)
    return df_base

In [68]:
test_set_dict

{'Exolos': {'1': [56736,
   {'1-0-(2, 2, 2, 0)-0-(2, 3, 3, 2)',
    '1-0-(1, 1, 0, 3)-0-(0, 1, 3, 0)',
    '1-0-(0, 1, 1, 0)-0-(2, 2, 2, 2)',
    '1-0-(0, 1, 1, 0)-0-(1, 3, 0, 1)',
    '1-0-(1, 0, 0, 0)-0-(1, 2, 3, 1)',
    '1-0-(3, 0, 0, 0)-0-(1, 0, 1, 3)',
    '1-0-(2, 0, 3, 0)-0-(0, 0, 0, 2)',
    '1-0-(1, 3, 1, 3)-0-(3, 1, 1, 0)',
    '1-0-(2, 1, 2, 0)-0-(1, 1, 2, 1)',
    '1-0-(1, 3, 1, 0)-0-(0, 0, 0, 1)',
    '1-0-(0, 0, 0, 0)-0-(1, 2, 0, 2)',
    '1-0-(3, 0, 2, 0)-0-(3, 3, 2, 0)',
    '1-0-(1, 1, 2, 2)-0-(1, 2, 1, 2)',
    '1-0-(0, 3, 2, 0)-0-(0, 2, 0, 2)',
    '1-0-(3, 0, 2, 1)-0-(0, 2, 0, 3)',
    '1-0-(0, 1, 3, 0)-0-(1, 3, 3, 1)',
    '1-0-(2, 0, 3, 0)-0-(1, 1, 3, 0)',
    '1-0-(3, 2, 2, 0)-0-(0, 0, 1, 3)',
    '1-0-(0, 0, 3, 1)-0-(1, 1, 2, 3)',
    '1-0-(1, 3, 3, 1)-0-(2, 3, 3, 3)',
    '1-0-(1, 3, 2, 1)-0-(1, 3, 1, 2)',
    '1-0-(2, 1, 2, 0)-0-(2, 2, 0, 0)',
    '1-0-(2, 2, 0, 3)-0-(0, 2, 0, 0)',
    '1-0-(3, 3, 3, 2)-0-(2, 3, 2, 2)',
    '1-0-(1, 1, 2, 0)-0-(1, 3, 1, 3)',


In [85]:
result

[]

In [34]:
teams = ["Exolos","6K","FUS","AB3","CHV","Contra","FUEGO","Cinta Negra"]

count = 0
for team in teams:
    week1_dict_to_df, week2_dict_to_df = create_position_dicts(team,test_set_dict,[1,9],9)
    df_g2 = df_with_positions(week1_dict_to_df,week2_dict_to_df,team)
    matches_columns = [col for col in df_g2.columns if 'match' in col] 
    df_melted = df_g2.melt(id_vars=['position'], value_vars=matches_columns, value_name='match').drop('variable', axis=1)
    temp_result = df_melted.groupby(['position', 'match']).size().reset_index(name='count')
    temp_result['team'] = team
    
    if count == 0:
         result_df = temp_result.copy()
         count += 1
    else:
        result_df = pd.concat([result_df,temp_result],ignore_index=True)

In [35]:
result_df.to_csv(path_or_buf='compilado.csv',index=False)

In [60]:
week1_dict_to_df, week2_dict_to_df = create_position_dicts('CHV',test_set_dict,[1,8],8)

In [65]:
df_g2 = df_with_positions(week1_dict_to_df,week2_dict_to_df,'CHV')

In [77]:
df_g2

,position,team,week1_match_1,week1_match_2,week1_match_3,week1_match_4,week2_match_1,week2_match_2,week2_match_3,week2_match_4
0,5,Cinta Negra,"(FUEGO, 0-2, FUS)","(AB3, 2-0, Exolos)","(CHV, 1-2, 6K)","(Contra, 0-2, Cinta Negra)","(CHV, 0-2, Cinta Negra)","(6K, 2-1, FUS)","(Contra, 1-2, Exolos)","(FUEGO, 1-2, AB3)"
1,5,Cinta Negra,"(FUEGO, 0-2, FUS)","(AB3, 1-2, Exolos)","(CHV, 0-2, 6K)","(Contra, 0-2, Cinta Negra)","(CHV, 1-2, Cinta Negra)","(6K, 0-2, FUS)","(Contra, 0-2, Exolos)","(FUEGO, 1-2, AB3)"
2,5,Cinta Negra,"(FUEGO, 1-2, FUS)","(AB3, 1-2, Exolos)","(CHV, 1-2, 6K)","(Contra, 1-2, Cinta Negra)","(CHV, 1-2, Cinta Negra)","(6K, 2-0, FUS)","(Contra, 2-0, Exolos)","(FUEGO, 0-2, AB3)"
3,5,Cinta Negra,"(FUEGO, 1-2, FUS)","(AB3, 2-1, Exolos)","(CHV, 1-2, 6K)","(Contra, 1-2, Cinta Negra)","(CHV, 1-2, Cinta Negra)","(6K, 0-2, FUS)","(Contra, 2-1, Exolos)","(FUEGO, 1-2, AB3)"
4,5,Cinta Negra,"(FUEGO, 0-2, FUS)","(AB3, 1-2, Exolos)","(CHV, 1-2, 6K)","(Contra, 0-2, Cinta Negra)","(CHV, 1-2, Cinta Negra)","(6K, 2-0, FUS)","(Contra, 0-2, Exolos)","(FUEGO, 1-2, AB3)"
...,...,...,...,...,...,...,...,...,...,...
23227,7,Cinta Negra,"(FUEGO, 2-0, FUS)","(AB3, 2-1, Exolos)","(CHV, 2-0, 6K)","(Contra, 1-2, Cinta Negra)","(CHV, 2-1, Cinta Negra)","(6K, 2-0, FUS)","(Contra, 1-2, Exolos)","(FUEGO, 2-1, AB3)"
23228,7,Cinta Negra,"(FUEGO, 0-2, FUS)","(AB3, 1-2, Exolos)","(CHV, 0-2, 6K)","(Contra, 1-2, Cinta Negra)","(CHV, 2-0, Cinta Negra)","(6K, 2-1, FUS)","(Contra, 0-2, Exolos)","(FUEGO, 0-2, AB3)"
23229,7,Cinta Negra,"(FUEGO, 1-2, FUS)","(AB3, 2-0, Exolos)","(CHV, 0-2, 6K)","(Contra, 0-2, Cinta Negra)","(CHV, 2-1, Cinta Negra)","(6K, 2-0, FUS)","(Contra, 1-2, Exolos)","(FUEGO, 2-0, AB3)"
23230,7,Cinta Negra,"(FUEGO, 2-1, FUS)","(AB3, 0-2, Exolos)","(CHV, 0-2, 6K)","(Contra, 0-2, Cinta Negra)","(CHV, 2-0, Cinta Negra)","(6K, 2-1, FUS)","(Contra, 0-2, Exolos)","(FUEGO, 1-2, AB3)"


In [81]:
matches_columns = [col for col in df_g2.columns if 'match' in col]

# Convertimos a un formato de dos columnas: 'position' y 'match'
df_melted = df_g2.melt(id_vars=['position'], value_vars=matches_columns, value_name='match').drop('variable', axis=1)

# Agrupamos por 'position' y 'match', y contamos las repeticiones
result = df_melted.groupby(['position', 'match']).size().reset_index(name='count')

# Mostramos el resultado
result['team'] = 'CHV'

In [82]:
result

,position,match,count,team
0,5,"(6K, 0-2, FUS)",512,CHV
1,5,"(6K, 1-2, FUS)",512,CHV
2,5,"(6K, 2-0, FUS)",512,CHV
3,5,"(6K, 2-1, FUS)",512,CHV
4,5,"(AB3, 0-2, Exolos)",512,CHV
...,...,...,...,...
77,7,"(FUEGO, 1-2, FUS)",2304,CHV
78,7,"(FUEGO, 2-0, AB3)",4096,CHV
79,7,"(FUEGO, 2-0, FUS)",4096,CHV
80,7,"(FUEGO, 2-1, AB3)",3904,CHV
